In [ ]:
from flask import Flask, request, jsonify, render_template_string
from joblib import load
import numpy as np

app = Flask(__name__)

# Load your trained model
model = load("model.pkl")

# HTML + CSS + JS
HTML = """
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Climate-Induced Disease Outbreak Prediction</title>
  <style>
    body { font-family: Arial, sans-serif; max-width: 600px; margin: auto; padding: 20px; background-color: #f0f8ff; }
    h1 { text-align: center; color: #0066cc; }
    label { display: block; margin-top: 10px; }
    input { width: 100%; padding: 8px; margin-top: 5px; }
    button { margin-top: 15px; padding: 10px 20px; background-color: #0066cc; color: white; border: none; cursor: pointer; }
    button:hover { background-color: #004999; }
    #result { margin-top: 20px; font-weight: bold; font-size: 18px; text-align: center; }
  </style>
</head>
<body>
  <h1>Climate-Induced Disease Outbreak Prediction</h1>
  <form id="predictForm">
    <label>Temperature (°C):</label>
    <input type="text" id="temp" placeholder="Enter temperature">
    
    <label>Humidity (%):</label>
    <input type="text" id="humidity" placeholder="Enter humidity">
    
    <label>Rainfall (mm):</label>
    <input type="text" id="rainfall" placeholder="Enter rainfall">
    
    <label>Air Quality (AQI):</label>
    <input type="text" id="air" placeholder="Enter air quality index">
    
    <button type="submit">Predict Disease Cases</button>
  </form>
  <div id="result"></div>

  <script>
    document.getElementById("predictForm").addEventListener("submit", async function(e) {
      e.preventDefault();

      const temp = document.getElementById("temp").value;
      const humidity = document.getElementById("humidity").value;
      const rainfall = document.getElementById("rainfall").value;
      const air = document.getElementById("air").value;

      if (!temp || !humidity || !rainfall || !air) {
        alert("Please enter all values!");
        return;
      }

      const response = await fetch("/predict", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ temp, humidity, rainfall, air })
      });

      const data = await response.json();
      document.getElementById("result").textContent =
        "Predicted disease cases: " + data.prediction;
    });
  </script>
</body>
</html>
"""

# Serve frontend
@app.route("/")
def home():
    return render_template_string(HTML)

# Predict using the trained model
@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    try:
        features = np.array([[float(data["temp"]),
                              float(data["humidity"]),
                              float(data["rainfall"]),
                              float(data["air"])]])
        prediction = model.predict(features)[0]
        prediction = round(prediction, 2)
    except Exception as e:
        return jsonify({"prediction": f"Error: {str(e)}"}), 400

    return jsonify({"prediction": prediction})

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Sep/2025 00:09:12] "GET / HTTP/1.1" 200 -
C:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
127.0.0.1 - - [15/Sep/2025 00:09:35] "POST /predict HTTP/1.1" 200 -
